## Corona-News Scraper from Tagesschau.de

In [41]:
import re
import requests
from bs4 import BeautifulSoup

import datetime
import pandas as pd

In [42]:
todays_date = datetime.datetime.now()
date_stamp = str(todays_date.date())

In [43]:
r = requests.get('https://www.tagesschau.de/')
soup = BeautifulSoup(r.text, "html5lib")

### Parse Tagesschau.de homepage for new articles

In [44]:
headlines = []
dachzeilen = []
teasers = []
links = []
texts = []
a = soup.find_all('a')
for item in a:
    if item.find('p', class_='dachzeile'):
        links.append(item.get('href'))
        texts.append(item.get_text())

In [45]:
# check the first text / link
print(texts[0])

print(links[0])



Coronavirus
Mehr Infektionen bei medizinischem Personal
Rund 6400 Ärzte und Pflegekräfte haben sich in Deutschland mit dem Coronavirus infiziert, acht sind bereits gestorben. Nicht alle Fälle sind bisher erfasst - laut NDR, WDR und SZ gibt es zudem eine große Dunkelziffer in der Altenpflege.
| mehr


/investigativ/ndr/corona-aerzte-pflegekraefte-101.html


### Check which articles on the homepage are about the Corona virus

In [46]:
# corona, covid19, coronavirus
corona_links = []
corona_texts = []
for i in range(len(texts)):
    if ("corona" in texts[i].lower()) | ("covid" in texts[i].lower()):
        if links[i].startswith("/"):
            corona_links.append(links[i])
            corona_texts.append(texts[i])

In [47]:
print(corona_links)
print("")
print(corona_texts)

['/investigativ/ndr/corona-aerzte-pflegekraefte-101.html', '/inland/bund-laender-beschluss-uebersicht-101.html', '/newsticker/liveblog-coronavirus-donnerstag-103.html', '/inland/krankenhaeuser-op-101.html', '/inland/interview-corona-pflegeheime-101.html', '/inland/corona-demonstrationsrecht-101.html', '/wirtschaft/standards-arbeitsplatz-kabinett-101.html', '/investigativ/swr/facebook-corona-103.html', '/ausland/siegesparade-russland-101.html', '/ausland/straferleichterungen-tuerkei-101.html', '/wirtschaft/boerse/arbeitslosigkeit-usa-103.html', '/ausland/grossbritannien-massnahmen-101.html', '/faktenfinder/coronavirus-wuhan-labor-101.html', '/ausland/kindersterblichkeit-corona-101.html', '/ausland/griechenland-fluechtlinge-243.html', '/ausland/seenotrettung-mittelmeer-107.html', '/ausland/coronavirus-karten-101.html']

['\nCoronavirus\nMehr Infektionen bei medizinischem Personal\nRund 6400 Ärzte und Pflegekräfte haben sich in Deutschland mit dem Coronavirus infiziert, acht sind bereits 

### Get full article for each news about corona virus

In [48]:
corona_articles = []
for link in corona_links:
    full_link = "https://www.tagesschau.de" + link
    print(full_link)
    html = requests.get(full_link)
    sub_soup = BeautifulSoup(html.text, "html5lib")
    article = sub_soup.find_all('p', class_='text small')
    paragraphs = []
    for i in range(len(article)):
        paragraphs.append(article[i].get_text())
    # unifying the paragraphs
    final_article = " ".join(paragraphs)
    corona_articles.append(final_article)
    

https://www.tagesschau.de/investigativ/ndr/corona-aerzte-pflegekraefte-101.html
https://www.tagesschau.de/inland/bund-laender-beschluss-uebersicht-101.html
https://www.tagesschau.de/newsticker/liveblog-coronavirus-donnerstag-103.html
https://www.tagesschau.de/inland/krankenhaeuser-op-101.html
https://www.tagesschau.de/inland/interview-corona-pflegeheime-101.html
https://www.tagesschau.de/inland/corona-demonstrationsrecht-101.html
https://www.tagesschau.de/wirtschaft/standards-arbeitsplatz-kabinett-101.html
https://www.tagesschau.de/investigativ/swr/facebook-corona-103.html
https://www.tagesschau.de/ausland/siegesparade-russland-101.html
https://www.tagesschau.de/ausland/straferleichterungen-tuerkei-101.html
https://www.tagesschau.de/wirtschaft/boerse/arbeitslosigkeit-usa-103.html
https://www.tagesschau.de/ausland/grossbritannien-massnahmen-101.html
https://www.tagesschau.de/faktenfinder/coronavirus-wuhan-labor-101.html
https://www.tagesschau.de/ausland/kindersterblichkeit-corona-101.ht

In [49]:
print(corona_articles)

['Rund 6400 Ärzte und Pflegekräfte haben sich in Deutschland mit dem Coronavirus infiziert, acht sind bereits gestorben. Nicht alle Fälle sind bisher erfasst - laut NDR, WDR und SZ gibt es zudem eine große Dunkelziffer in der Altenpflege.  Von Antonius Kempmann, Reiko Pinkert, NDR, und Martin Kaul, WDR  6395 Mitarbeiterinnen und Mitarbeiter in Krankenhäusern und Arztpraxen sind in Deutschland mit dem Coronavirus infiziert, acht von ihnen sind bereits verstorben. Das meldet das Robert Koch-Institut (RKI) in seinem Lagebericht vom Donnerstag. Damit hat sich die Zahl der gemeldeten Infektionen in dieser Berufsgruppe innerhalb der letzten zwei Wochen beinahe verdreifacht.  Vor zwei Wochen hatte das RKI erstmals die Zahl von 2300 infizierten Klinikmitarbeitern bekannt gegeben, gleichzeitig aber darauf hingewiesen, dass es eine Dunkelziffer gebe, weil manche Gesundheitsämter aus Kapazitätsgründen auf die Berufsgruppe bezogene Zahlen nicht weiterleiteten.  Eine von NDR, WDR und "Süddeutscher 

In [50]:
len(corona_articles)

17

### Make into CSV file

In [51]:
columns = ["date", "url", "article", "resort"]
df = pd.DataFrame(columns=columns)
df["url"] = corona_links
df["article"] = corona_articles
df["date"] = todays_date
df["resort"] = df["url"].apply(lambda x: x.split("/")[1])

In [52]:
df.head()

,date,url,article,resort
0,2020-04-16 21:54:37.967161,/investigativ/ndr/corona-aerzte-pflegekraefte-...,Rund 6400 Ärzte und Pflegekräfte haben sich in...,investigativ
1,2020-04-16 21:54:37.967161,/inland/bund-laender-beschluss-uebersicht-101....,Die Länderchefs haben mit der Kanzlerin zwar E...,inland
2,2020-04-16 21:54:37.967161,/newsticker/liveblog-coronavirus-donnerstag-10...,Die Länderregierungen legen den Bund-Länder-Be...,newsticker
3,2020-04-16 21:54:37.967161,/inland/krankenhaeuser-op-101.html,150.000 Betten auf Normalstationen sind derzei...,inland
4,2020-04-16 21:54:37.967161,/inland/interview-corona-pflegeheime-101.html,Gerade ältere Menschen in Pflegeheimen treffen...,inland


### Export CSV

In [53]:
df.to_csv("data/" + date_stamp)